In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp drive/My\ Drive/UCF101/NumPy.rar Numpy.rar

In [0]:
!unrar x Numpy.rar

In [0]:
import os
import sys
import numpy as np
import pickle

In [0]:
data = pickle.load(open('Data.p', 'rb'))

In [0]:
num_categories, folder_names, Xtrain, ytrain, Xtest, ytest = data

In [0]:
dirs = [int(file.split('_')[1].split('.')[0]) for file in os.listdir('NumPy/')]
for i in range(len(list(num_categories.keys()))):
    for j in range(num_categories[list(num_categories.keys())[i]]):
        if 200*i+j not in dirs:
            print(200*i+j)

In [0]:
Xtrain.shape[0] + Xtest.shape[0]

13320

### Data Flow Pipeline

In [0]:
Xtrain[:10], ytrain[:10], Xtest[:5], ytest[:5]

(array([ 9266,  4040, 18716, 13011, 16095, 11113, 12847, 10038, 16489,
         6218], dtype=int32),
 array([46, 20, 93, 65, 80, 55, 64, 50, 82, 31], dtype=int32),
 array([15848,  7415,  8750, 12338,  1846], dtype=int32),
 array([79, 37, 43, 61,  9], dtype=int32))

In [0]:
def load_sample(num, time_step = 64):
    npy = np.load('NumPy/Features_'+str(num)+'.npy')
    shape = npy.shape[0]
    out = []
    for i in range(time_step):
        out.append(npy[int((shape*(i/time_step))//1)])
    out = np.array(out)
    return out

In [0]:
def create_batch(num, X_t, batch_size=64, time_step=64):
    X = []
    batch = X_t[num:num+batch_size]
    for i in batch:
        X.append(load_sample(i, time_step))
    X = np.array(X)
    return X

In [0]:
def create_data(X_t, batch_size=64, time_step=64):
    X_train = []
    for num in range(0, X_t.shape[0]-batch_size, batch_size):
        X_train.append(create_batch(num, X_t, batch_size, time_step))
    return np.array(X_train)

In [0]:
X_train = create_data(Xtrain).reshape((187*64,64,512))

In [0]:
X_test = create_data(Xtest).reshape((20*64,64,512))

In [0]:
X_train.shape, X_test.shape

((11968, 64, 512), (1280, 64, 512))

### Model

In [0]:
import numpy as np
import sys
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as K
from keras.layers import Input, Dense, Dropout, LSTM
from keras.models import Model
from keras.utils import to_categorical
import pickle

In [0]:
y_train = to_categorical(ytrain, 101)
y_test = to_categorical(ytest, 101)
y_train.shape, y_test.shape

((11988, 101), (1332, 101))

In [0]:
def rnn(input_shape):
    input1 = Input(input_shape)

    X = LSTM(768, dropout=0.3, return_sequences=True)(input1)
    X = LSTM(512, dropout=0.3)(X)
    X = Dense(1024, activation = 'relu')(X)
    X = Dropout(0.3)(X)
    X = Dense(101, activation='softmax')(X)

    return Model(input1, X)


In [0]:
Rnn = rnn((64,512))
Rnn.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 64, 512)           0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 64, 768)           3935232   
_________________________________________________________________
lstm_17 (LSTM)               (None, 512)               2623488   
_________________________________________________________________
dense_19 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_9 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 101)               103525    
Total params: 7,187,557
Trainable params: 7,187,557
Non-trainable params: 0
________________________________________________

In [0]:
Rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
history = Rnn.fit(X_train, y_train[:11968], batch_size=512, epochs=10, validation_split=0.1)

In [0]:
Rnn.evaluate(X_test, y_test[:1280])

1280/1280 [==============================] - 5s 4ms/step


[0.2785111678298563, 0.92421875]

In [0]:
Rnn.save('Epoch100.h5')